In [3]:
# Nikhil Patil
# CSEC 620
# Project 1 New Model
'''
this model is far more efficient than my previous one. 
It uses the same dataset, but it uses the clean_df function from the dataprep library to clean the dataset as well all 77 features rather than just 4.  
The larger sequential keras model versus functional keras model aids in efficiency as well. 

'''

# Import the necessary libraries
import time
import warnings
import numpy as np
import pandas as pd
from dataprep.clean import clean_df
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


warnings.filterwarnings("ignore") # ignore warnings

dataset_csv_path = 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv' # the dataset used for this model. 

# Load and clean the dataset 
df = pd.read_csv(dataset_csv_path)
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace("/", "", regex=False) 
df = clean_df(df,data_type_detection = "none", standardize_missing_values = "remove") 
'''
The clean_df function is used to clean the dataset. The data_type_detection parameter is set to "none" because of the way that clean_df returns the dataframe.
standardize_missing_values is set to "remove" to remove rows with missing values.
'''

# create list of features
features = []
for column in df.columns:
    features.append(column)

df.replace([np.inf, -np.inf], np.nan, inplace=True) # replace infinite values with Not a Number (NaN)
df.fillna(0, inplace=True) # fill values with 0 

df = df[features]
df_features = df.drop('label', axis=1) # drop the label column
df_labels = df['label'] # get the label column


# Begin to fit the model
scaler = StandardScaler() 
transformed_features = scaler.fit_transform(df_features)

# Encode the Label column
encoder = OneHotEncoder(sparse_output=False)
labels_encoded = encoder.fit_transform(df_labels.values.reshape(-1,1))

# Use train_test_split from scikit-learn to split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(transformed_features, labels_encoded, train_size= .8, test_size=0.2)

# Create the model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(labels_encoded.shape[1], activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
start_time = time.perf_counter()
model.fit(X_train, y_train, epochs=10, batch_size=32)
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
end_time = time.perf_counter()

print(f"Loss: {loss: .8f}, Accuracy: {accuracy: .12f}")
print(f"Time taken: {end_time - start_time:.2f} seconds")


Column Headers Cleaning Report:
	77 values cleaned (97.47%)
Number of Entries Cleaning Report:
	4 entries dropped (0.0%)
Downcast Memory Report:
	Memory reducted from 174241739 to 98844245. New size: (56.73%)
Epoch 1/3
5644/5644 [==============================] - 27s 5ms/step - loss: 0.0168 - accuracy: 0.9964
Epoch 2/3
5644/5644 [==============================] - 25s 4ms/step - loss: 0.0101 - accuracy: 0.9986
Epoch 3/3
1411/1411 [==============================] - 5s 3ms/step - loss: 0.0042 - accuracy: 0.9991
Loss:  0.004238, Accuracy:  1.00
Time taken:  82.63 seconds
